# Imports


In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

# AWS credentials

In [0]:
dbutils.fs.ls("/FileStore/tables")

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# Mounting S3 Bucket

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0ae9e110c9db-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/databricks-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[72]: True

In [0]:
display(dbutils.fs.ls("/mnt/databricks-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/databricks-bucket/topics/0ae9e110c9db.geo/,0ae9e110c9db.geo/,0,1704233853334
dbfs:/mnt/databricks-bucket/topics/0ae9e110c9db.pin/,0ae9e110c9db.pin/,0,1704233853334
dbfs:/mnt/databricks-bucket/topics/0ae9e110c9db.user/,0ae9e110c9db.user/,0,1704233853334


# Transferring S3 Bucket to Dataframes

In [0]:
def topics_to_dataframe():
    spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")
    
    topics = [".pin", ".geo", ".user"]
    df_array = []
    # File location and type
    # Asterisk(*) indicates reading all the content of the specified file that have .json extension
    for file_extension in topics:
        file_location = "/mnt/databricks-bucket/topics/0ae9e110c9db" + file_extension + "/partition=0/*.json" 
        file_type = "json"
        # Ask Spark to infer the schema
        infer_schema = "true"
        # Read in JSONs from mounted S3 bucket
        df = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)
        # Display Spark dataframe to check its content
        display(df)
        df_array.append(df)
    return df_array

In [0]:
df_array = topics_to_dataframe()
df_array

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
home-decor,"Традиционные шведские коттеджи, обычно с красным фасадом — это настоящее воплощением идеального зимнего уюта. Они обычно оформлены очень просто и ✌PUFIK. Beautiful Interiors. On…",1,136k,https://i.pinimg.com/originals/32/eb/72/32eb72e4fd8654c115a64528bd1f34b4.png,6717,image,PUFIK Interiors & Inspirations,Local save in /data/home-decor,"Scandinavian Cottage,Swedish Cottage,Swedish Home Decor,Swedish Farmhouse,Swedish Style,Swedish Kitchen,Kitchen Black,Swedish House,Cozy Cottage",〚 Уютные шведские коттеджи от Carina Olander 〛 ◾ Фото ◾ Идеи ◾ Дизайн,bc5ab9ee-505e-44f6-92ba-677fe4fdf3e3
home-decor,"Традиционные шведские коттеджи, обычно с красным фасадом — это настоящее воплощением идеального зимнего уюта. Они обычно оформлены очень просто и ✌PUFIK. Beautiful Interiors. On…",1,136k,https://i.pinimg.com/originals/32/eb/72/32eb72e4fd8654c115a64528bd1f34b4.png,6717,image,PUFIK Interiors & Inspirations,Local save in /data/home-decor,"Scandinavian Cottage,Swedish Cottage,Swedish Home Decor,Swedish Farmhouse,Swedish Style,Swedish Kitchen,Kitchen Black,Swedish House,Cozy Cottage",〚 Уютные шведские коттеджи от Carina Olander 〛 ◾ Фото ◾ Идеи ◾ Дизайн,bc5ab9ee-505e-44f6-92ba-677fe4fdf3e3
christmas,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",1,5k,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,1706,image,Wear24-7,Local save in /data/christmas,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,b5c8a1b5-9e90-4522-9bec-2477b698d5b7
christmas,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",1,5k,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,1706,image,Wear24-7,Local save in /data/christmas,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,b5c8a1b5-9e90-4522-9bec-2477b698d5b7
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holiday

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01 02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03 05:41:59
British Virgin Islands,2293,-87.7946,-159.647,2022-03-21 10:46:53
Saint Kitts and Nevis,10663,-27.3474,-162.83,2019-07-25 18:53:51
Antigua and Barbuda,7922,-88.0974,-172.052,2021-01-27 09:14:19
Antigua and Barbuda,8606,-88.0974,-172.052,2021-03-28 14:54:07
Netherlands Antilles,603,14.0083,-141.603,2019-06-25 05:13:01
Dominican Republic,9979,14.9967,-120.682,2018-07-18 19:01:46


age,date_joined,first_name,ind,last_name
27,2016-03-08 13:38:37,Christopher,2015,Bradshaw
20,2015-12-01 15:08:31,Christopher,5076,Butler
39,2017-07-19 07:12:04,Michelle,7790,Gutierrez
49,2016-04-22 20:36:02,Brittany,10509,Thompson
21,2015-11-10 09:27:42,Andrea,8731,Alexander
21,2015-11-10 09:27:42,Andrea,8731,Alexander
24,2016-03-31 20:56:39,Austin,8887,Rodriguez
23,2015-12-01 18:15:02,Christine,7768,Cortez
36,2015-12-20 16:38:13,Michelle,4315,Prince
36,2015-12-20 16:38:13,Michelle,4315,Prince


Out[75]: [DataFrame[category: string, description: string, downloaded: bigint, follower_count: string, image_src: string, index: bigint, is_image_or_video: string, poster_name: string, save_location: string, tag_list: string, title: string, unique_id: string],
 DataFrame[country: string, ind: bigint, latitude: double, longitude: double, timestamp: string],
 DataFrame[age: bigint, date_joined: string, first_name: string, ind: bigint, last_name: string]]

# Unmounting S3 Bucket

In [0]:
dbutils.fs.unmount("/mnt/databricks-bucket/topics/")